In [ ]:
!pip install -r ../requirements.txt

In [1]:
import sys
sys.path.insert(0, "../")

# Downloading and processing data

There are two ways you can obtain OSM data in osm.pbf format:  
- Download it yourself: [parts of the world](https://download.geofabrik.de/), [cities](https://download.bbbike.org/osm/bbbike/), [adjustable area](https://extract.bbbike.org/) (via mail), [adjustable area](https://export.hotosm.org/en/v3/) (online), [planet](https://planet.maps.mail.ru/pbf/)
- Let the program download it for you

If the map is downloaded you can specify the filename:

In [ ]:
from visibility_graph import VisibilityGraph

map_data = VisibilityGraph()
filename = "../maps/kozlovo.osm.pbf"
bbox = [36.2, 56.5, 36.7, 56.7]
map_data.compute_geometry(bbox=bbox, filename=filename)

Or, alternatively, you can only specify the bounding box, and the map will be downloaded automatically ([curl](https://curl.se/) & [osmosis](https://wiki.openstreetmap.org/wiki/Osmosis) required):

In [ ]:
bbox = [34, 59, 34.2, 59.1]
map_data.compute_geometry(bbox=bbox)

Data inside this area can be processed using VisibilityGraph with chosen or default parameters.  
If not specified, optimal parameters will be computed by the algorithm.

In [ ]:
map_data.build_dataframe(epsilon_polygon=0.003,
                         epsilon_linestring=0.001,
                         bbox_comp=10)

Computed data can also be saved in .h5 file to skip data processing the next time:

In [ ]:
map_data.save_geometry("../maps/user_area.h5")

# Using precomputed data and building visibility graph

In [4]:
from visibility_graph import VisibilityGraph

map_data = VisibilityGraph()
map_data.load_geometry("../maps/user_area.h5")

Visibility graph can be built and (optionally) saved as networkx graph and (optionally) visualised using mplleaflet:

In [5]:
%%time
import mplleaflet

map_plot=('r', {0: "royalblue", 1: "r", 2: "k"})
G, fig = map_data.build_graph(inside_percent=0.4,
                              graph=True,
                              map_plot=map_plot)

print('edges: ', G.number_of_edges())
print('nodes: ', G.number_of_nodes())
mplleaflet.display(fig=fig)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  2.95it/s]


edges:  1152
nodes:  191
Wall time: 9.35 s


VisibilityGraph may also be used to find incident edges for a single point.  
This feature is used for pathfinding without graph building:

In [ ]:
import matplotlib.pyplot as plt
import mplleaflet

start = ([36.35, 56.57], None, None, None, None)
incidents = map_data.incident_vertices(start)

fig = plt.figure()
plt.scatter(start[0][0], start[0][1], color='r')
for p in incidents:
    plt.scatter(p[0][0], p[0][1], color='b')
mplleaflet.display(fig=fig)